In [1]:
import json
import jinja2
import nestedtext as nt
import os
import re
import openpyxl
AV_bench_env = jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath='/projects/H3A0/workspaces/jdavid/h3v1_release/UVM_tools/uvm_analog_tb/AV_bench/templates/'))
print ("hello world")

hello world


In [2]:
regs_wb = openpyxl.load_workbook('H3A0_regmap_v1.03.xlsx')
sheetnames = regs_wb.sheetnames
print(sheetnames)

['History_', 'InnoLink_SerDes', 'SYSCLK', 'RFPLL', 'RX', 'OBSTX', 'Masterbias', 'DPA', 'Chip Monitor', 'LO and LO Test Port']


In [3]:
top_registers = {"regs":{}}
for sheet in regs_wb.sheetnames:
    print (sheet)
    headerrow = None
    name_col = None
    descr_col = None
    defaults_col = None
    on_state_col = None
    width_col = None
    for row in regs_wb[sheet].iter_rows(max_row=5):
        for cell in row:
            if cell.value == "Block":
                headerrow = cell.row
                print (f'{sheet} headercell = ',headerrow, cell.col_idx, cell.row)
            if cell.value == 'RFA Top Pin Name':
                name_col = cell.col_idx-1 
            if cell.value == 'Description':
                descr_col = cell.col_idx-1 
            if cell.value == "OFF state":
                defaults_col = cell.col_idx-1
                on_state_col = cell.col_idx
            if cell.value == 'Width':
                width_col = cell.col_idx-1 
        if headerrow:
            break
    if headerrow and name_col and defaults_col and width_col:
        for row in regs_wb[sheet].iter_rows(min_row=headerrow+1):
            #print (row[width_col].value)
            try:
                regmatch = re.match(r'^\s*(?P<name>\w[\w\d_]+)(?:<(?P<msb>\d+):(?P<lsb>\d+)>)?\s*$',row[name_col].value)
            except:
                break    
            if regmatch:
                top_registers['regs'][regmatch.group('name')] = {
                    'width': row[width_col].value,
                    'default': ''.join(["'b",str(row[defaults_col].value)]),
                    'sheet': sheet
                    }
                if row[descr_col].value:
                    descr_match = re.match(r'(\S.*\S)',str(row[descr_col].value))
                    if descr_match:
                        top_registers['regs'][regmatch.group('name')]['description'] = descr_match.group(0)
                if regmatch.group('msb'):
                    top_registers['regs'][regmatch.group('name')]['msb'] = regmatch.group('msb')
                    top_registers['regs'][regmatch.group('name')]['lsb'] = regmatch.group('lsb')
                if isinstance(row[on_state_col].value, int):
                    if row[on_state_col].value != row[defaults_col].value:
                        top_registers['regs'][regmatch.group('name')]['on_value']= f"'b{str(row[on_state_col].value)}"
                        #=  ''.join(["'b",str(row[block_on_col].value)])        pass
            else: 
                print ('no match', row[name_col].value)


print ( json.dumps(top_registers,indent=4))

History_
InnoLink_SerDes
InnoLink_SerDes headercell =  2 1 2
SYSCLK
SYSCLK headercell =  2 1 2
RFPLL
RFPLL headercell =  2 1 2
RX
RX headercell =  2 1 2
OBSTX
OBSTX headercell =  2 1 2
Masterbias
Masterbias headercell =  2 1 2
DPA
DPA headercell =  2 1 2
Chip Monitor
Chip Monitor headercell =  2 1 2
LO and LO Test Port
LO and LO Test Port headercell =  2 1 2
{
    "regs": {
        "il_cm_cal_comp_out_digo": {
            "width": 1,
            "default": "'bR",
            "sheet": "InnoLink_SerDes"
        },
        "il_cm_dig_intf_resetb_digi": {
            "width": 1,
            "default": "'b0",
            "sheet": "InnoLink_SerDes",
            "on_value": "'b1"
        },
        "il_cm_ext_reg_en_digi": {
            "width": 1,
            "default": "'b0",
            "sheet": "InnoLink_SerDes",
            "description": "Enable",
            "on_value": "'b1"
        },
        "il_cm_ext_reg_lv_en_digi": {
            "width": 1,
            "default": "'b0",
        

In [4]:

dut_spec = nt.load("rfa_top_spec.nt", top='dict')


In [5]:
#check for register existence add description default, and on value if they exist 
reg_extract = re.compile(r'(?P<blockid>(?P<block>(?:obstx_adc|[A-Za-z]+))\d?[A-Za-z]?)_(?P<functn>\w.*)_(?P<direction>dig[io])$')
dut_spec['regs'] = {}
dut_spec['reg_blocks'] = {}
for port in dut_spec['ports']:
    if dut_spec['ports'][port]['porttype'] in ['register']:
        reg_mo = re.match(reg_extract,port)
        if reg_mo:
            register_name = '_'.join([reg_mo.group('block'),reg_mo.group('functn'),reg_mo.group('direction')])
            block_id = reg_mo.group('blockid')
            if register_name in top_registers['regs']:
                dut_spec['ports'][port]['reg_info'] = {
                    'name': register_name, 
                    'blockid': block_id,
                    'width': top_registers['regs'][register_name]['width'],
                    'sheet': top_registers['regs'][register_name]['sheet']
                    }
                if dut_spec['ports'][port]['direction'] in ['input']:
                    dut_spec['ports'][port]['reg_info']['default'] = top_registers['regs'][register_name]['default']
                    if top_registers['regs'][register_name].get('on_value'):
                        dut_spec['ports'][port]['reg_info']['on_value'] = top_registers['regs'][register_name]['on_value']
                if top_registers['regs'][register_name].get('description'):
                    dut_spec['ports'][port]['reg_info']['description'] = top_registers['regs'][register_name]['description']
                
                if dut_spec['ports'][port].get('msb') != top_registers['regs'][register_name].get('msb'):
                    print (f"MSB mismatch {port} msb={dut_spec['ports'][port]['msb']} {register_name} msb={top_registers['regs'][register_name]['msb']}" )
                if register_name in dut_spec['regs']:
                    dut_spec['regs'][register_name]['blocks'].append(reg_mo.group('blockid'))
                else:
                    dut_spec['regs'][register_name] = {'blocks': [reg_mo.group('blockid')]}
                if not block_id in dut_spec['reg_blocks']:
                    dut_spec['reg_blocks'][block_id] = {'regs':{}, 'ro_regs': {}}
                if dut_spec['ports'][port]['direction'] in ['input']:
                    dut_spec['reg_blocks'][block_id]['regs'][port] =  {'default':top_registers['regs'][register_name]['default']}
                    if top_registers['regs'][register_name].get('on_value'):
                        dut_spec['reg_blocks'][block_id]['regs'][port]['on_value'] = top_registers['regs'][register_name]['on_value']
                else:
                    dut_spec['reg_blocks'][block_id]['ro_regs'] = port                        
                
            else:
                print(register_name, 'for port',port, "is not found")
            #print (reg_mo.groupdict())
        else:
            print ("no pattern match in ",port)
passed_regs = 0
failed_regs = 0
for reg in top_registers['regs']:
    if reg in dut_spec['regs'].keys():
        passed_regs += 1
    else:
        print (reg, "has no ports")
        failed_regs +=1
if failed_regs: 
    print (f'ERROR: {failed_regs} registers did not have ports, - {passed_regs} did')
else:
    print (f'{failed_regs} registers did not have ports, - {passed_regs} did')
    
print ( json.dumps(dut_spec,indent=4))
with open('dut_regs_spec.json', 'w') as f:
    json.dump(dut_spec, f, indent=4)

0 registers did not have ports, - 911 did
{
    "library": "H3HA0",
    "name": "rfa_top",
    "ports": {
        "AMUX": {
            "direction": "output",
            "porttype": "analog"
        },
        "ANALOG_HS_TEST1A_N": {
            "direction": "output",
            "porttype": "digital"
        },
        "ANALOG_HS_TEST1A_P": {
            "direction": "output",
            "porttype": "digital"
        },
        "ANALOG_HS_TEST1B_N": {
            "direction": "output",
            "porttype": "digital"
        },
        "ANALOG_HS_TEST1B_P": {
            "direction": "output",
            "porttype": "digital"
        },
        "ANALOG_HS_TEST2_N": {
            "direction": "output",
            "porttype": "digital"
        },
        "ANALOG_HS_TEST2_P": {
            "direction": "output",
            "porttype": "digital"
        },
        "AVDD1P2_IL0": {
            "direction": "input",
            "porttype": "power",
            "supply_nom": "1.2",
  